# Exercise 

In [27]:
import pydataset 
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.preprocessing import OneHotEncoder


In [82]:
tips = pydataset.data('tips')

In [83]:
tips

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
240,29.03,5.92,Male,No,Sat,Dinner,3
241,27.18,2.00,Female,Yes,Sat,Dinner,2
242,22.67,2.00,Male,Yes,Sat,Dinner,2
243,17.82,1.75,Male,No,Sat,Dinner,2


### 1. Load the tips dataset.
#### A. Create a column named price_per_person. This should be the total bill divided by the party size.

In [84]:
tips['price_per_person'] = round(tips['total_bill'] / tips['size'], 2)

In [85]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,8.49
2,10.34,1.66,Male,No,Sun,Dinner,3,3.45
3,21.01,3.50,Male,No,Sun,Dinner,3,7.00
4,23.68,3.31,Male,No,Sun,Dinner,2,11.84
5,24.59,3.61,Female,No,Sun,Dinner,4,6.15


In [87]:
tips.isna().sum()

total_bill          0
tip                 0
sex                 0
smoker              0
day                 0
time                0
size                0
price_per_person    0
dtype: int64

#### b. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

In [ ]:
### I think that total_bill and time will be important predicting the tip amount

#### c. Use select k best to select the top 2 features for predicting tip amount. What are they?

In [88]:
enc = OneHotEncoder(handle_unknown='ignore')


In [89]:
cat_cols = tips.drop(columns = [ 'tip','total_bill', 'size', 'price_per_person'])

cat_cols

,sex,smoker,day,time
1,Female,No,Sun,Dinner
2,Male,No,Sun,Dinner
3,Male,No,Sun,Dinner
4,Male,No,Sun,Dinner
5,Female,No,Sun,Dinner
...,...,...,...,...
240,Male,No,Sat,Dinner
241,Female,Yes,Sat,Dinner
242,Male,Yes,Sat,Dinner
243,Male,No,Sat,Dinner


In [91]:
enc.fit(cat_cols)


OneHotEncoder(handle_unknown='ignore')

In [92]:
encoded_array = enc.transform(cat_cols).toarray()

In [93]:
encoded_df = pd.DataFrame(encoded_array, columns=enc.get_feature_names())
encoded_df

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,x0_Female,x0_Male,x1_No,x1_Yes,x2_Fri,x2_Sat,x2_Sun,x2_Thur,x3_Dinner,x3_Lunch
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
239,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
240,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
241,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
242,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [94]:
tips = pd.concat([tips, encoded_df], axis=1).drop(columns=cat_cols)
tips.head()

,total_bill,tip,size,price_per_person,x0_Female,x0_Male,x1_No,x1_Yes,x2_Fri,x2_Sat,x2_Sun,x2_Thur,x3_Dinner,x3_Lunch
1,16.99,1.01,2.0,8.49,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,10.34,1.66,3.0,3.45,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,21.01,3.50,3.0,7.00,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,23.68,3.31,2.0,11.84,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
5,24.59,3.61,4.0,6.15,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [99]:
mask = tips.isna()
nan_rows = mask.any(axis=1)
tips[nan_rows]

,total_bill,tip,size,price_per_person,x0_Female,x0_Male,x1_No,x1_Yes,x2_Fri,x2_Sat,x2_Sun,x2_Thur,x3_Dinner,x3_Lunch
244,18.78,3.0,2.0,9.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [104]:
tips = tips.dropna()
tips.isna().sum()

total_bill          0
tip                 0
size                0
price_per_person    0
x0_Female           0
x0_Male             0
x1_No               0
x1_Yes              0
x2_Fri              0
x2_Sat              0
x2_Sun              0
x2_Thur             0
x3_Dinner           0
x3_Lunch            0
dtype: int64

In [137]:
X = tips.drop(columns='tip')
y = tips['tip']

In [112]:
# make the thing
kbest = SelectKBest(f_regression, k=2)
# fit the thing
_ = kbest.fit(X, y)

In [114]:
X.iloc[:,kbest.get_support()].head()

,total_bill,size
1,16.99,2.0
2,10.34,3.0
3,21.01,3.0
4,23.68,2.0
5,24.59,4.0


#### d. Use recursive feature elimination to select the top 2 features for tip amount. What are they?

In [115]:
model = LinearRegression()

In [116]:
rfe = RFE(model, n_features_to_select=2)

In [117]:
# fit the RFE:
rfe.fit(X, y)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [118]:
rfe.ranking_

array([ 1,  7,  3, 10,  4,  6, 11, 12,  9,  8,  1,  2,  5])

In [121]:
pd.DataFrame({
    'rfe_ranking': rfe.ranking_}, index=X.columns)

,rfe_ranking
total_bill,1
size,7
price_per_person,3
x0_Female,10
x0_Male,4
x1_No,6
x1_Yes,11
x2_Fri,12
x2_Sat,9
x2_Sun,8


##### Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?

In [ ]:
# Not sure

## Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [135]:
def select_kbest(x, y, k):
    kbest = SelectKBest(f_regression, k=k)
    # fit the thing
    _ = kbest.fit(X, y)
    return X.iloc[:,kbest.get_support()].head()
    

In [138]:
select_kbest(X, y, 2)

,total_bill,size
1,16.99,2.0
2,10.34,3.0
3,21.01,3.0
4,23.68,2.0
5,24.59,4.0


 ## Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [143]:
def rfe(X, y, num_features):
    model = LinearRegression()
    rfe = RFE(model, n_features_to_select=num_features)
    rfe.fit(X, y)
    return pd.DataFrame({
        'rfe_ranking': rfe.ranking_}, index=X.columns)
    

In [125]:
rfe(X, y, 2)

,rfe_ranking
total_bill,1
size,7
price_per_person,3
x0_Female,10
x0_Male,4
x1_No,6
x1_Yes,11
x2_Fri,12
x2_Sat,9
x2_Sun,8


## Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).


In [126]:
swiss = pydataset.data('swiss')

In [127]:
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [139]:
X = swiss.drop(columns='Fertility')
y = swiss['Fertility']

In [141]:
select_kbest(X, y, 3)

,Examination,Education,Catholic
Courtelary,15,12,9.96
Delemont,6,9,84.84
Franches-Mnt,5,5,93.40
Moutier,12,7,33.77
Neuveville,17,15,5.16


In [144]:
rfe(X, y, 3)

,rfe_ranking
Agriculture,2
Examination,1
Education,1
Catholic,3
Infant.Mortality,1
